In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.utils import to_categorical

In [ ]:
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm
import gc
import math

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / (1024 ** 2)} MB")
else:
    print("No GPU found.")

In [ ]:
df=pd.read_csv("/kaggle/input/imabalance/emotion_sentimen_dataset.csv")

In [ ]:
df1=df

In [ ]:
df=df.loc[(df['Emotion'] == 'hate') | (df['Emotion'] == 'neutral')]

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
im_ratio=len(df.loc[(df['Emotion']=='neutral')])/len(df.loc[(df['Emotion']=='hate')])

In [ ]:
len(df.loc[(df['Emotion']=='hate')])

In [ ]:
len(df.loc[(df['Emotion']=='neutral')])

In [ ]:
1/im_ratio

In [ ]:
df=df[df['Emotion']!='empty']

In [ ]:
a1=df["Emotion"].unique()
l=[]
for i in a1:
    l.append(sum(df["Emotion"]==i))

In [ ]:
from matplotlib import pyplot as plt
fig=plt.figure(figsize=(20,8))
plt.bar(a1,l)
plt.show()

In [ ]:
df = df.drop_duplicates(subset='text')

In [ ]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_Y,test_Y=train_test_split(df['text'],df['Emotion'],test_size=0.30,random_state=42)
train_X,valid_X,train_Y,valid_Y=train_test_split(train_X,train_Y,test_size=0.30,random_state=42)

In [ ]:
#lebel encode for y
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(['hate','neutral'])

In [ ]:
train_Y=label_encoder.transform(train_Y)
valid_Y=label_encoder.transform(valid_Y)

In [ ]:
L123=[0,1]

In [ ]:
original_train_Y = label_encoder.inverse_transform(L123)

In [ ]:
original_train_Y

In [ ]:
import tensorflow as tf

In [ ]:
# using tenser flow
import transformers
model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
tokenizer = transformers.TFBertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def vectorcreator(data):
    new_data=[[0 for i in range(768)]]
    for i in range(0,len(data),1000): 
        with tf.device('/GPU:0'):
            sentence_1 = tokenizer(data[i:min(i+1000,len(data))])
            embeddings_1 = model.bert.embeddings(input_ids=sentence_1["input_ids"])
            new_data= tf.concat([new_data,tf.reduce_mean(embeddings_1,axis=1)],0)
    
    return new_data[1:len(new_data)]

In [ ]:
d1=vectorcreator(train_X)

In [ ]:
v1=vectorcreator(valid_X)

In [ ]:
t1=vectorcreator(test_X)

In [ ]:
train_data=d1.numpy()

In [ ]:
valid_data=v1.numpy()

In [ ]:
test_data=t1.numpy()

In [ ]:
print(len(train_Y[train_Y==0]))
print(len(train_Y[train_Y==1]))

In [ ]:
train_Y=train_Y.reshape(train_Y.shape[0],)

In [ ]:
train_data.shape

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote_create_1=SMOTE(random_state=42)
senti_resample_X,senti_resample_Y=smote_create_1.fit_resample(train_data,train_Y)

In [ ]:
a1=len(senti_resample_Y[senti_resample_Y==1])
b1=len(senti_resample_Y[senti_resample_Y==0])

In [ ]:
print(str(a1)+"  "+str(b1))

In [ ]:
# Check the size (number of elements)
size = senti_resample_X.size
print(f"Number of elements: {size}")

# Check the memory usage
memory_usage = senti_resample_X.nbytes
print(f"Memory usage: {memory_usage} bytes")

In [ ]:
train_re_X,che_X,train_re__Y,che_Y=train_test_split(senti_resample_X,senti_resample_Y,test_size=0.70,random_state=42)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=53) # Reduce to 2 dimensions 
# Fit and transform the data 
X_pca = pca.fit_transform(senti_resample_X)

In [ ]:
train_re_X.shape

In [ ]:
a1=len(train_re__Y[train_re__Y==1])
b1=len(train_re__Y[train_re__Y==0])

In [ ]:
print(str(a1)+"  "+str(b1))

In [ ]:
import cuml
import cuml.svm
import cupy as cp
import numpy as np
import pandas as pd

In [ ]:
senti_resample_X.dtype

In [ ]:
X_gpu = cp.asarray(senti_resample_X) 
y_gpu = cp.asarray(senti_resample_Y)

In [ ]:
from sklearn.svm import SVC

In [ ]:
smote_svm = SVC(kernel='linear') 
# Train the classifier 
smote_svm.fit(train_re_X,train_re__Y)

In [ ]:
import joblib
joblib.dump(smote_svm, 'model_smote_senti')

In [ ]:
pred_Y=smote_svm.predict(valid_data)

In [ ]:
from sklearn.metrics import classification_report
rp=classification_report(valid_Y,pred_Y,target_names=['0','1'])
print(rp)

In [ ]:
pred_y=smote_svm.predict(test_data)
test_p_y=label_encoder.inverse_transform(pred_y)
rp=classification_report(test_Y,test_p_y,target_names=['hate','nutral'])
print(rp)

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score
precision = precision_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None) 
recall = recall_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None) 
for label, precision in zip(['hate', 'neutral'], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip(['hate', 'neutral'], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
h,n=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None)
micro_f1=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='micro')
macro_f1=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='macro')
wei_f1=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='weighted')
print(f"hate:{h}  neutral:{n}")
print(f"micro:{micro_f1}")
print(f"macro:{macro_f1}")
print(f"weighted:{wei_f1}")

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(test_Y,test_p_y).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
confusion_matrix(test_Y,test_p_y)

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(label_encoder.transform(test_Y),label_encoder.transform(test_p_y))
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
# Assuming senti_resample_Y is your target labels
# Convert your target labels to one-hot encoded format
senti_resample_Y_1 = to_categorical(senti_resample_Y, num_classes=2)

# Similarly, convert your validation labels
valid_Y_1 = to_categorical(valid_Y, num_classes=2)

# Define the model
model = Sequential([
    Dense(512, input_dim=768, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Use softmax for multi-class classification
])

# Compile the model
model.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])
early_stopping = EarlyStopping(monitor='val_f1_score', patience=5, verbose=1, restore_best_weights=True)
# Train the model
model.fit(senti_resample_X, senti_resample_Y_1, epochs=40, batch_size=32, validation_data=(valid_data, valid_Y_1), callbacks=[early_stopping])


In [ ]:
from tensorflow.keras.models import load_model 
# Load the H5 model file 
model = load_model('/kaggle/input/smotesenti/other/default/1/smote-model.h5')

In [ ]:
pred_Y=model.predict(valid_data).argmax(axis=1)

In [ ]:
pred_Y.shape

In [ ]:
valid_Y

In [ ]:
from sklearn.metrics import classification_report
rp=classification_report(valid_Y,pred_Y,target_names=['0','1'])
print(rp)

In [ ]:
pred_y=model.predict(test_data).argmax(axis=1)
test_p_y=label_encoder.inverse_transform(pred_y)
rp=classification_report(test_Y,test_p_y,target_names=['hate','nutral'])
print(rp)

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score
precision = precision_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None) 
recall = recall_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None) 
for label, precision in zip(['hate', 'neutral'], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip(['hate', 'neutral'], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
h,n=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None)
micro_f1=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='micro')
macro_f1=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='macro')
wei_f1=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='weighted')
print(f"hate:{h}  neutral:{n}")
print(f"micro:{micro_f1}")
print(f"macro:{macro_f1}")
print(f"weighted:{wei_f1}")

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(test_Y,test_p_y).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(label_encoder.transform(test_Y),label_encoder.transform(test_p_y))
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
def distance(s_line,train_data,feature,intercept):
    print(train_data.shape)
    deno=math.sqrt(np.sum(np.square(s_line)))
    dist=(abs((np.matmul(train_data[:,0:feature],s_line)+intercept))/deno)
    return dist

In [ ]:
def data_cre(traing_data_X=None,train_data_Y=None,clas=0,c1=0.50,cn1=0.50):
    train_data=np.copy(traing_data_X)
    #np.place(train_data[:, 768],train_data[:, 768]!=clas, [0])
    #np.place(train_data[:, 768],train_data[:, 768]==clas, [1])
    train_data_1= train_data[train_data_Y ==clas ]
    train_data_non_1=train_data[ train_data_Y!=clas ]
    length_1=train_data_1.shape[0]
    length_non_1=train_data_non_1.shape[0]
    feature=train_data_1.shape[1]
    center_1=train_data_1[:,:].mean(axis=0).reshape(1,feature)
    center_non_1=train_data_non_1[:,:].mean(axis=0).reshape(1,feature)
    line_1=np.array([center_1,center_non_1]).reshape(2,feature)
    Y_1=[1,0]
    li_1=svm.SVC(kernel='linear')
    li_1.fit(line_1,Y_1)
    d_1=distance(li_1.coef_[0].reshape(feature,1),train_data_1[:,0:feature],feature,li_1.intercept_)
    d_non_1=distance(li_1.coef_[0].reshape(feature,1),train_data_non_1[:,0:feature],feature,li_1.intercept_)
    sorted_indices_1=np.argsort(d_1,axis=0)
    train_data_1=train_data_1[sorted_indices_1]
    sorted_indices_non_1=np.argsort(d_non_1,axis=0)
    train_data_non_1=train_data_non_1[sorted_indices_non_1]
    l1=train_data_1.shape[0]
    ln1=train_data_non_1.shape[0]
    y=[clas]*(math.ceil(c1*l1))+[1-clas]*math.ceil(cn1*ln1)
    y=np.array(y)
    train_data_1=train_data_1.reshape(l1,feature)
    train_data_non_1=train_data_non_1.reshape(ln1,feature)
    f_t= np.append(train_data_1[0:math.ceil(c1*l1),:], train_data_non_1[0:math.ceil(cn1*ln1),:], axis=0)
    indices = np.arange(f_t.shape[0]) 
    np.random.shuffle(indices)
    f_t=f_t[indices]
    y=y[indices]
    indices = np.arange(f_t.shape[0]) 
    np.random.shuffle(indices)
    f_t=f_t[indices]
    y=y[indices]
    return f_t,y

In [ ]:
sp_1=svm_im(traing_data_X=train_data,train_data_Y=train_Y,clas=0,c1=0.09,cn1=0.2)

In [ ]:
import joblib 
# Load the model 
loaded_model = joblib.load('/kaggle/input/svm-imbalance/other/default/1/model_bi_im.joblib')

In [ ]:
sp_1

In [ ]:
i1=20
for i in range(20,40):
    print(i1)
    pt_1=model_1[i].predict(valid_data)
    print(classification_report(valid_Y, pt_1, target_names=['class0','class1']))
    i1=i1+1

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
confusion_matrix(valid_Y, pt_1)

In [ ]:
print(classification_report(valid_Y, pt_1, target_names=['class0','class1']))

In [ ]:
testing_x=vectorcreator(test_X)

In [ ]:
testing_x=testing_x.numpy()

In [ ]:
sp_1=model_1[26]

In [ ]:
test_1=sp_1.predict(testing_x)

In [ ]:
test_1=test_1.astype(int)

In [ ]:
test_1=label_encoder.inverse_transform(test_1)

In [ ]:
rp=classification_report(test_Y,test_1,target_names=['hate','nutral'])
print(rp)

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

In [ ]:
precision = precision_score(test_Y,test_1,labels=['hate', 'neutral'], average=None) 
recall = recall_score(test_Y,test_1,labels=['hate', 'neutral'], average=None) 
for label, precision in zip(['hate', 'neutral'], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip(['hate', 'neutral'], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
f1_scores

In [ ]:
f1_score(test_Y,test_1,labels=['hate', 'neutral'], average='micro') 

In [ ]:
f1_score(test_Y,test_1,labels=['hate', 'neutral'], average='weighted') 

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
tn, fp, fn, tp = confusion_matrix(test_Y,test_1).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

In [ ]:
fpr, tpr, _ = roc_curve(label_encoder.transform(test_Y),label_encoder.transform(test_1))
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

In [ ]:
# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
from sklearn.metrics import f1_score
import cuml 
from cuml.svm import SVC 
from sklearn.datasets import make_classification 
from sklearn.model_selection import train_test_split 
import cupy as cp

In [ ]:
tr=np.array(trin)
d1=pd.DataFrame(tr,columns=['Majority', 'Minority', 'Macro F1 score'])
d1.to_excel('train_cur_last.xlsx')

In [ ]:
tr=np.array(val)
d1=pd.DataFrame(tr,columns=['Majority', 'Minority', 'Macro F1 score'])
d1.to_excel('val_cur1_last.xlsx')

In [ ]:
trin=[]
val=[]
model_1=[]
for i in range(35,101,5):
    for j in range(30,34,5):
        class_1=i/100
        class_n_1=j/100
        train_x,train_y=data_cre(traing_data_X=d1,train_data_Y=train_Y,clas=1,c1=class_1,cn1=class_n_1)
        print(len(train_y[train_y==1]))
        print(len(train_y[train_y==0]))
        # Define the model
        sp_1=svm.SVC(kernel='linear',C=1.0)
        # Fit the model on the training data
        sp_1.fit(train_x,train_y)
        model_1.append(sp_1)
        y_1=sp_1.predict(d1)
        macro_f1_1 = f1_score(train_Y, y_1, average='macro')
        trin.append([class_1,class_n_1,macro_f1_1])
        y_1=sp_1.predict(valid_data)
        macro_f1_2 = f1_score(valid_Y, y_1, average='macro')
        val.append([class_1,class_n_1,macro_f1_2])
        print(f"c1:{class_1} cn1:{class_n_1} test:{macro_f1_1} valid:{macro_f1_2}")

apply ANN

In [ ]:
train_x,train_y=data_cre(traing_data_X=train_data,train_data_Y=train_Y,clas=0,c1=0.32,cn1=0.17)

In [ ]:
len(train_y[train_y==1])

In [ ]:
len(train_y[train_y==0])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.utils import to_categorical

In [ ]:
# Assuming senti_resample_Y is your target labels
# Convert your target labels to one-hot encoded format
senti_Y_1 = to_categorical(train_y, num_classes=2)

# Similarly, convert your validation labels
valid_Y_1 = to_categorical(valid_Y, num_classes=2)

# Define the model
model1 = Sequential([
    Dense(512, input_dim=768, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Use softmax for multi-class classification
])

# Compile the model
model1.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])

early_stopping = EarlyStopping(monitor='val_f1_score', patience=5, verbose=1, restore_best_weights=True)
# Train the model
history=model1.fit(train_x,senti_Y_1,epochs=38, batch_size=100, validation_data=(valid_data, valid_Y_1), callbacks=[early_stopping])


In [ ]:
pred_Y=model1.predict(valid_data).argmax(axis=1)

In [ ]:
from tensorflow.keras.models import load_model 
# Load the H5 model file 
model1 = load_model('/kaggle/input/my_model_senti/other/default/1/my_model.h5')

In [ ]:
pred_y=model1.predict(test_data).argmax(axis=1)

In [ ]:
model1.save('my_model.h5')

In [ ]:
test_p_y=label_encoder.inverse_transform(pred_y)

In [ ]:
test_p_y

In [ ]:
from sklearn.metrics import classification_report
rp=classification_report(test_Y,test_p_y,target_names=['hate','neutral'])
print(rp)

In [ ]:
rp=classification_report(test_Y,test_p_y,target_names=['neutral','hate'])
print(rp)

In [ ]:
precision = precision_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None) 
recall = recall_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None) 
for label, precision in zip(['hate', 'neutral'], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip(['hate', 'neutral'], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
micr=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='micro')
print(f"Micro f1 score:{micr}")
macr=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='macro')
print(f"Macro f1 score:{macr}")
weg=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average='weighted')
print(f"Weight f1 score:{weg}")
a1,b1=f1_score(test_Y,test_p_y,labels=['hate', 'neutral'], average=None) 
print(f"hate:{a1} neutral:{b1}")

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(test_Y,test_p_y).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(label_encoder.transform(test_Y),label_encoder.transform(test_p_y))
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()



In [ ]:
train=[]
val=[]

In [ ]:
df1 = pd.DataFrame(train, columns=['major', 'minor','macro_f1_train','macro_f1_val'])


In [ ]:
df1.to_excel('ANN_sentiment_1.xlsx', index=False)

In [ ]:
for i1 in range(5,101,5):
    for j1 in range(17,18,5):
        i=i1/100
        j=j1/100
        print(f"minority:{i} and majority:{j}")
        train_x,train_y=data_cre(traing_data_X=train_data,train_data_Y=train_Y,clas=0,c1=i,cn1=j)
        senti_Y_1 = to_categorical(train_y, num_classes=2)
        # Similarly, convert your validation labels
        valid_Y_1 = to_categorical(valid_Y, num_classes=2)
        model1 = Sequential([
        Dense(512, input_dim=768, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # Use softmax for multi-class classification
        ])

        # Compile the model
        model1.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])
        early_stopping = EarlyStopping(monitor='val_f1_score', patience=9, verbose=1, restore_best_weights=True)        
        history=model1.fit(train_x,senti_Y_1,epochs=10, batch_size=100, validation_data=(valid_data, valid_Y_1),callbacks=[early_stopping])
        train_p_y_1=model1.predict(train_data).argmax(axis=1)
        t_f1=f1_score(train_Y,train_p_y_1,labels=[0, 1], average='macro')
        val_p_y_1=model1.predict(valid_data).argmax(axis=1)
        val_f1=f1_score(valid_Y,val_p_y_1,labels=[0, 1], average='macro')
        train.append([i,j,t_f1,val_f1])
        print(train[-1])

In [ ]:
conf_matrix = confusion_matrix(valid_Y, pred_Y) # Print the confusion matrix 
print(conf_matrix) 

In [ ]:
conf_matrix= confusion_matrix(test_p_y,test_Y)
print(conf_matrix)

In [ ]:
valid_Y_1.shape

In [ ]:
from joblib import load
m1=load("/kaggle/input/svm-imbalance/other/default/1/model_bi_im.joblib")

In [ ]:
pred_test=m1.predict(testing_x)

In [ ]:
pred_test1=list(map(int,pred_test))

In [ ]:
pred_test1=label_encoder.inverse_transform(pred_test1)

In [ ]:
print(classification_report(test_Y, pred_test1, target_names=['hate', 'neutral']))

In [ ]:
from sklearn import metrics

In [ ]:
# Calculate AUC
auc = metrics.roc_auc_score(valid_Y, pt_1)
print(f"AUC: {auc}")

In [ ]:
p_1=m_1.predict(valid_data)

In [ ]:
valid_1=np.copy(valid_Y)

In [ ]:
valid_1

In [ ]:
#valid_1[valid_1!=6]=0

In [ ]:
#valid_1[valid_1==6]=1

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
conf_matrix = confusion_matrix(valid_1, pt_1)


In [ ]:
conf_matrix

In [ ]:
print(precision_score(valid_1,p_1,average=None))

In [ ]:
print(recall_score(valid_1,p_1,average=None))

In [ ]:
conf_matrix1 = confusion_matrix(train_Y, pt_1)

In [ ]:
conf_matrix1

In [ ]:
train_Y

In [ ]:
data_credit.head(5)

SPAM DETECTION

In [ ]:
import pandas as pd

In [ ]:
data_spam=pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='ISO-8859-1')

In [ ]:
data_spam.head()

In [ ]:
len(data_spam[data_spam['v1']=='ham'])

In [ ]:
len(data_spam[data_spam['v1']=='spam'])

In [ ]:
d=len(data_spam[data_spam['v1']=='ham'])/len(data_spam[data_spam['v1']=='spam'])

In [ ]:
d

In [ ]:
len(data_spam['Unnamed: 2'].unique())

In [ ]:
a1=data_spam['v1'].unique()

In [ ]:
l=[]
for i in a1:
    l.append(sum(data_spam['v1']==i))

In [ ]:
from matplotlib import pyplot as plt
fig=plt.figure(figsize=(20,8))
plt.bar(a1,l)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
spam_train_X,spam_test_X,spam_train_Y,spam_test_Y=train_test_split(data_spam['v2'],data_spam['v1'],test_size=0.30,random_state=42)
spam_train_X,spam_valid_X,spam_train_Y,spam_valid_Y=train_test_split(spam_train_X,spam_train_Y,test_size=0.30,random_state=42)

In [ ]:
#lebel encode for y
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
train_Y=label_encoder.fit_transform(spam_train_Y)

In [ ]:
valid_Y=label_encoder.transform(spam_valid_Y)

In [ ]:
train_Y.shape

In [ ]:
from collections import Counter
Counter(train_Y)

In [ ]:
d1=vectorcreator(spam_train_X)

In [ ]:
d1=d1.numpy()

In [ ]:
len(d1[train_Y==1])

In [ ]:
d1.shape[1]

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote_create=SMOTE(random_state=42)

In [ ]:
spam_resample_X,spam_resample_Y=smote_create.fit_resample(d1,train_Y)

In [ ]:
from collections import Counter

In [ ]:
Counter(spam_resample_Y)

In [ ]:
from sklearn import svm

In [ ]:
gen_svm=svm.SVC(kernel='linear',C=1.0,probability=True)

In [ ]:
gen_svm.fit(spam_resample_X,spam_resample_Y)

In [ ]:
test_prd=gen_svm.predict(t1)
pre_our=label_encoder.inverse_transform(test_prd)
print(classification_report(spam_test_Y, pre_our, target_names=['ham','spam']))

In [ ]:
precision = precision_score(spam_test_Y, pre_our, labels=['ham','spam'], average=None) 
recall = recall_score(spam_test_Y, pre_our, labels=['ham','spam'], average=None) 
for label, precision in zip(['ham','spam'], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip(['ham','spam'], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
print("Micro F1 score")
print(f1_score(spam_test_Y, pre_our,labels=['ham','spam'], average='micro')) 
print("weight F1 score")
print(f1_score(spam_test_Y, pre_our,labels=['ham','spam'], average='weighted'))
print("Macro F1 score")
print(f1_score(spam_test_Y, pre_our,labels=['ham','spam'], average='macro'))

In [ ]:
print(f1_score(spam_test_Y, pre_our,labels=['ham','spam'],average=None))

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(spam_test_Y, pre_our).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(label_encoder.transform(spam_test_Y),label_encoder.transform(pre_our))
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

ANN

In [ ]:
# Assuming senti_resample_Y is your target labels
# Convert your target labels to one-hot encoded format
test_re_Y_1 = to_categorical(spam_resample_Y, num_classes=2)

# Similarly, convert your validation labels
valid_Y_1 = to_categorical(valid_Y, num_classes=2)

# Define the model
model1 = Sequential([
    Dense(512, input_dim=768, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Use softmax for multi-class classification
])

# Compile the model
model1.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])

early_stopping = EarlyStopping(monitor='val_f1_score', patience=40, verbose=1, restore_best_weights=True)
# Train the model
history=model1.fit(spam_resample_X,test_re_Y_1,epochs=40, batch_size=100, validation_data=(v1, valid_Y_1), callbacks=[early_stopping])


In [ ]:
v1=vectorcreator(spam_valid_X)

In [ ]:
pre_gen=gen_svm.predict(v1)

In [ ]:
pre_gen=label_encoder.inverse_transform(pre_gen)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(spam_valid_Y, pre_gen, target_names=a1))

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix_spam_gen = confusion_matrix(spam_valid_Y, pre_gen)

In [ ]:
conf_matrix_spam_gen

In [ ]:
import joblib

In [ ]:
joblib.dump(gen_svm,'SMOTE-SVM')

In [ ]:
joblib.dump(svm_tun,'our-svm-spam')

ANN test

In [ ]:
val_pred_y=model1.predict(v1).argmax(axis=1)

In [ ]:
pre_gen=label_encoder.inverse_transform(val_pred_y)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(spam_valid_Y, pre_gen, target_names=['spam','ham']))

In [ ]:
t1=vectorcreator(spam_test_X)

In [ ]:
test_pred_y=model1.predict(t1).argmax(axis=1)

In [ ]:
pre_test=label_encoder.inverse_transform(test_pred_y)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(spam_test_Y, pre_test, target_names=['spam','ham']))

In [ ]:
model1.save('spam-ham.h5')

In [ ]:
precision = precision_score(spam_test_Y, pre_test,labels=['spam', 'ham'], average=None) 
recall = recall_score(spam_test_Y, pre_test,labels=['spam', 'ham'], average=None) 
for label, precision in zip(['spam', 'ham'], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip(['spam', 'ham'], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
micr=f1_score(spam_test_Y, pre_test,labels=['spam', 'ham'], average='micro')
print(f"Micro f1 score:{micr}")
macr=f1_score(spam_test_Y, pre_test,labels=['spam', 'ham'], average='macro')
print(f"Micro f1 score:{macr}")
weg=f1_score(spam_test_Y, pre_test,labels=['spam', 'ham'], average='weighted')
print(f"Weight f1 score:{weg}")
a1,b1=f1_score(spam_test_Y, pre_test,labels=['spam', 'ham'], average=None) 
print(f"spam:{a1} ham:{b1}")


In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(spam_test_Y, pre_test).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(label_encoder.transform(spam_test_Y),label_encoder.transform(pre_test))
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()


OUR MODEL

Counter({0: 3372, 1: 528})

In [ ]:
label_encoder.inverse_transform([0,1])

In [ ]:
ft,y=data_cre(traing_data_X=d1,train_data_Y=train_Y,clas=0,c1=0.9,cn1=0.75)
print(len(y[y==0]))
print(len(y[y==1]))       
svm_tun=svm.SVC(kernel='linear',C=5,probability=True)
svm_tun.fit(ft,y)
pred=svm_tun.predict(v1)
pre_our=label_encoder.inverse_transform(pred)
print(classification_report(spam_valid_Y, pre_our, target_names=['ham','spam']))

In [ ]:
svm_tun=joblib.load("/kaggle/input/spam_ham/other/default/1/our-svm-spam")

In [ ]:
test_prd=svm_tun.predict(t1)
pre_our=label_encoder.inverse_transform(test_prd)
print(classification_report(spam_test_Y, pre_our, target_names=['ham','spam']))

In [ ]:
precision = precision_score(spam_test_Y, pre_our,labels=['ham','spam'], average=None) 
recall = recall_score(spam_test_Y, pre_our,labels=['ham','spam'], average=None) 
for label, precision in zip(['ham','spam'], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip(['ham','spam'], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
print("micro_F1")
print(f1_score(spam_test_Y, pre_our,labels=['ham','spam'], average='micro'))
print("average F1")
print(f1_score(spam_test_Y, pre_our,labels=['ham','spam'], average='weighted'))
print("macro F1")
print(f1_score(spam_test_Y, pre_our,labels=['ham','spam'], average='macro')) 

In [ ]:
print(f1_score(spam_test_Y, pre_our,labels=['ham','spam'], average=None)) 

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(spam_test_Y, pre_our).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(label_encoder.transform(spam_test_Y),label_encoder.transform(pre_our))
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
import joblib
joblib.dump(svm_tun, 'spam_ham_tune_org.joblib')

Our model ANN

In [ ]:
train_x,train_y=data_cre(traing_data_X=d1,train_data_Y=train_Y,clas=1,c1=0.9,cn1=0.2)

In [ ]:
len(train_y[train_y==1])

In [ ]:
len(train_y[train_y==0])

In [ ]:
# Assuming senti_resample_Y is your target labels
# Convert your target labels to one-hot encoded format
train_Y_1 = to_categorical(train_y, num_classes=2)

# Similarly, convert your validation labels
valid_Y_1 = to_categorical(valid_Y, num_classes=2)

# Define the model
model_o_1 = Sequential([
    Dense(512, input_dim=768, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Use softmax for multi-class classification
])

# Compile the model
model_o_1.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])

early_stopping = EarlyStopping(monitor='val_f1_score', patience=40, verbose=1, restore_best_weights=True)
# Train the model
history=model_o_1.fit(train_x,train_Y_1,epochs=100, batch_size=100, validation_data=(v1, valid_Y_1), callbacks=[early_stopping])


In [ ]:
val_pred_o_y=model_o_1.predict(v1).argmax(axis=1)

In [ ]:
pre_o_gen=label_encoder.inverse_transform(val_pred_o_y)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(spam_valid_Y, pre_o_gen, target_names=['spam','ham']))

In [ ]:
test_pred_y=model_o_1.predict(t1).argmax(axis=1)


In [ ]:
pre_o_test=label_encoder.inverse_transform(test_pred_y)

In [ ]:
m1=model_o_1

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(spam_test_Y, pre_o_test, target_names=['spam','ham']))

In [ ]:
precision = precision_score(spam_test_Y, pre_o_test,labels=['spam', 'ham'], average=None) 
recall = recall_score(spam_test_Y, pre_o_test,labels=['spam', 'ham'], average=None) 
for label, precision in zip(['spam', 'ham'], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip(['spam', 'ham'], recall):
    print(f'Recall for {label}: {recall:.4f}')


In [ ]:
micr=f1_score(spam_test_Y, pre_o_test,labels=['spam', 'ham'], average='micro')
print(f"Micro f1 score:{micr}")
macr=f1_score(spam_test_Y, pre_o_test,labels=['spam', 'ham'], average='macro')
print(f"Macro f1 score:{macr}")
weg=f1_score(spam_test_Y, pre_o_test,labels=['spam', 'ham'], average='weighted')
print(f"Weight f1 score:{weg}")
a1,b1=f1_score(spam_test_Y, pre_o_test,labels=['spam', 'ham'], average=None) 
print(f"spam:{a1} ham:{b1}")

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(spam_test_Y, pre_o_test).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(label_encoder.transform(spam_test_Y),label_encoder.transform(pre_o_test))
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
con=confusion_matrix(spam_valid_Y, pre_our)

In [ ]:
con

In [ ]:
train_Y

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
train=[]
valis=[]
for i in range(5,101,5):
    for j in range(20,21,5):
        c1=i/100
        cn1=j/100
        ft,y=data_cre(traing_data_X=d1,train_data_Y=train_Y,clas=1,c1=c1,cn1=cn1)
        gen_svm=svm.SVC(kernel='linear',C=1.0,probability=True)
        gen_svm.fit(ft,y)
        y_1=gen_svm.predict(train_data)
        macro_f1_1 = f1_score(train_Y, y_1, average='macro')
        train.append([c1,cn1,macro_f1_1])
        y_1=gen_svm.predict(v1)
        macro_f1_2 = f1_score(valid_Y, y_1, average='macro')
        valis.append([c1,cn1,macro_f1_2])
        print(f"c1:{c1} cn1:{cn1} test:{macro_f1_1} valid:{macro_f1_2}")

In [ ]:
t1=np.array(train)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
train=[]
valis=[]
for i in range(90,91,5):
    for j in range(5,101,5):
        c1=i/100
        cn1=j/100
        ft,y=data_cre(traing_data_X=d1,train_data_Y=train_Y,clas=1,c1=c1,cn1=cn1)
        # Assuming senti_resample_Y is your target labels
        # Convert your target labels to one-hot encoded format
        train_Y_1 = to_categorical(y, num_classes=2)
        # Similarly, convert your validation labels
        valid_Y_1 = to_categorical(valid_Y, num_classes=2)

        # Define the model
        model_o_1 = Sequential([
            Dense(512, input_dim=768, activation='relu'),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(2, activation='softmax')  # Use softmax for multi-class classification
        ])

        # Compile the model
        model_o_1.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])

        early_stopping = EarlyStopping(monitor='val_f1_score', patience=9, verbose=1, restore_best_weights=True)
        # Train the model
        history=model_o_1.fit(ft,train_Y_1,epochs=10, batch_size=100, validation_data=(v1, valid_Y_1), callbacks=[early_stopping])

        y_1=model_o_1.predict(d1).argmax(axis=1)
        macro_f1_1 = f1_score(train_Y, y_1, average='macro')
        y_1=model_o_1.predict(v1).argmax(axis=1)
        macro_f1_2 = f1_score(valid_Y, y_1, average='macro')
        train.append([c1,cn1,macro_f1_1,macro_f1_2])
        print(f"c1:{c1} cn1:{cn1} test:{macro_f1_1} valid:{macro_f1_2}")

In [ ]:
df = pd.DataFrame(train[0:20], columns=['macro', 'micro', 'test_f1_score','val_f1_score'])

In [ ]:
df

In [ ]:
df.to_excel('curve_2.xlsx', index=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Generate some example validation data
majority_class_percen = t1[:,0:1]
minority_class_percen = t1[:,1:2]
macro_f1=t1[:,2:3]  # Example loss values for demonstration
print(len(macro_f1))
# Create a meshgrid for 3D plotting
X, Y = np.meshgrid(majority_class_percen, minority_class_percen)
Z = macro_f1

# Create a 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Add labels and title
ax.set_xlabel('majority_class_percen')
ax.set_ylabel('minority_class_percen')
ax.set_zlabel('macro_f1')
ax.set_title('3D train prediction')

# Add a color bar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Show the plot
plt.show()


In [ ]:
t1=np.array(valis)

In [ ]:
t1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Generate some example validation data
majority_class_percen = t1[:,0:1]
minority_class_percen = t1[:,1:2]
macro_f1=t1[:,2:3]  # Example loss values for demonstration
print(len(macro_f1))
# Create a meshgrid for 3D plotting
X, Y = np.meshgrid(majority_class_percen, minority_class_percen)
Z = macro_f1

# Create a 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Add labels and title
ax.set_xlabel('majority_class_percen')
ax.set_ylabel('minority_class_percen')
ax.set_zlabel('macro_f1')
ax.set_title('3D train prediction')

# Add a color bar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Show the plot
plt.show()

Credit Card fraud

In [ ]:
!pip install cuml

In [ ]:
import cuml
from cuml import preprocessing
from cuml.svm import SVC
from cuml.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
import numpy as np

In [ ]:
data_credit=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [ ]:
len(data_credit[data_credit['Class']==0])

In [ ]:
len(data_credit[data_credit['Class']==1])

In [ ]:
len(data_credit[data_credit['Class']==0])/len(data_credit[data_credit['Class']==1])

In [ ]:
data_credit.shape

In [ ]:
data_credit['Class']=data_credit['Class'].astype('int32')

In [ ]:
a1=data_credit['Class'].unique()
l=[]
for i in a1:
    l.append(sum(data_credit['Class']==i))

In [ ]:
from matplotlib import pyplot as plt
fig=plt.figure(figsize=(20,8))
plt.bar(a1,l)
plt.show()

In [ ]:
data_credit.shape

In [ ]:
len(data_credit[data_credit['Class']==1])

In [ ]:
data_credit_clean=data_credit.drop_duplicates()

In [ ]:
data_credit.shape

In [ ]:
data_credit=data_credit_clean.astype(np.float32)

In [ ]:
data_credit_X=data_credit.iloc[:,0:30].values

In [ ]:
data_credit_Y=data_credit.iloc[:,30].values

In [ ]:
l=data_credit_Y.shape[0]
data_credit_Y.reshape(l,)

In [ ]:
data_credit_Y=data_credit_Y.astype('int32')

In [ ]:
from sklearn import  preprocessing

In [ ]:
scaler = preprocessing.StandardScaler()
data_credit_X = scaler.fit_transform(data_credit_X)

In [ ]:
from sklearn.model_selection import train_test_split
credit_train_X,credit_test_X,credit_train_Y,credit_test_Y=train_test_split(data_credit_X,data_credit_Y,test_size=0.30,random_state=42)
credit_train_X,credit_valid_X,credit_train_Y,credit_valid_Y=train_test_split(credit_train_X,credit_train_Y,test_size=0.30,random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote_create=SMOTE(random_state=42)

In [ ]:
credit_resample_X,credit_resample_Y=smote_create.fit_resample(credit_train_X,credit_train_Y)

In [ ]:
unique_elements, counts = np.unique(credit_resample_X, return_counts=True)

In [ ]:
from collections import Counter

In [ ]:
Counter(credit_resample_Y)

ANN

In [ ]:
# Assuming senti_resample_Y is your target labels
# Convert your target labels to one-hot encoded format
train_Y_1 = to_categorical(credit_resample_Y, num_classes=2)

# Similarly, convert your validation labels
valid_Y_1 = to_categorical(credit_valid_Y, num_classes=2)

# Define the model
model_c_1 = Sequential([
    Dense(15, input_dim=30, activation='relu'),
    Dense(8, activation='relu'),
    Dense(2, activation='softmax')  # Use softmax for multi-class classification
])

# Compile the model
model_c_1.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])

early_stopping = EarlyStopping(monitor='val_f1_score', patience=40, verbose=1, restore_best_weights=True)
# Train the model
history=model_c_1.fit(credit_resample_X,train_Y_1,epochs=100, batch_size=100, validation_data=(credit_valid_X,valid_Y_1), callbacks=[early_stopping])


In [ ]:
y_pred = model_c_1.predict(credit_valid_X).argmax(axis=1)
# Generate the classification report 
report = classification_report(credit_valid_Y, y_pred,target_names=['0','1'])

In [ ]:
print(report)

In [ ]:
y_t_pred = model_c_1.predict(credit_test_X).argmax(axis=1)
# Generate the classification report 
report = classification_report(credit_test_Y, y_t_pred,target_names=['0','1'])

In [ ]:
print(report)

In [ ]:
model_c_1.save("credit_1.h5")

In [ ]:
precision = precision_score(credit_test_Y, y_t_pred,labels=[0,1], average=None) 
recall = recall_score(credit_test_Y, y_t_pred,labels=[0,1], average=None) 
for label, precision in zip([0,1], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip([0,1], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
micr=f1_score(credit_test_Y, y_t_pred,labels=[0,1], average='micro')
print(f"Micro f1 score:{micr}")
macr=f1_score(credit_test_Y, y_t_pred,labels=[0,1], average='macro')
print(f"Macro f1 score:{macr}")
weg=f1_score(credit_test_Y, y_t_pred,labels=[0,1], average='weighted')
print(f"Weight f1 score:{weg}")
a1,b1=f1_score(credit_test_Y, y_t_pred,labels=[0,1], average=None) 
print(f"0:{a1} 1:{b1}")

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(credit_test_Y, y_t_pred).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(credit_test_Y,y_t_pred)
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
# Check shapes 
print(f"Shape of credit_resample_X: {credit_resample_X.shape}") 
print(f"Shape of credit_resample_Y: {credit_resample_Y.shape}")

In [ ]:
smote_m=SVC(kernel='rbf', C=9, gamma=0.1,cache_size=2000, tol=1e-6, max_iter=1000)
# Fit the model on the training data
smote_m.fit(credit_resample_X,credit_resample_Y)
# Predict on the test data 
y_pred = smote_m.predict(credit_valid_X) 
# Generate the classification report 
report = classification_report(credit_valid_Y, y_pred,target_names=['0','1'])
#print(f"c : {9}  and gamma : {gamma}")
print("Classification Report:\n",report)

In [ ]:
from joblib import dump
dump(smote_m, 'SMOTE_credit.joblib')

In [ ]:
y_pred = smote_m.predict(credit_test_X) 
# Generate the classification report 
report = classification_report(credit_test_Y, y_pred,target_names=['0','1'])

In [ ]:
print(report)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
precision = precision_score(credit_test_Y, y_pred,labels=[0,1], average=None) 
recall = recall_score(credit_test_Y, y_pred,labels=[0,1], average=None) 
for label, precision in zip([0,1], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip([0,1], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:

print("micro_F1")
print(f1_score(credit_test_Y, y_pred,labels=[0,1], average='micro'))
print("average F1")
print(f1_score(credit_test_Y, y_pred,labels=[0,1], average='weighted'))
print("macro F1")
print(f1_score(credit_test_Y, y_pred,labels=[0,1], average='macro')) 

print(f1_score(credit_test_Y, y_pred,labels=[0,1], average=None))

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(credit_test_Y, y_pred).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(credit_test_Y,y_pred)
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Define different values for C and gamma
C_values = [0.1, 1, 10]
gamma_values = [0.001, 0.01, 0.1]

# Iterate over different combinations of C and gamma
for C in range(1,10):
    for gamma in  gamma_values:
        # Create a SVC model
        smote_m=SVC(kernel='rbf', C=C, gamma=gamma,cache_size=2000, tol=1e-6, max_iter=1000)
        # Fit the model on the training data
        smote_m.fit(credit_resample_X,credit_resample_Y)
        # Predict on the test data 
        y_pred = smote_m.predict(credit_valid_X) 
        # Generate the classification report 
        report = classification_report(credit_valid_Y, y_pred,target_names=['0','1'])
        print(f"c : {C}  and gamma : {gamma}")
        print("Classification Report:\n",report)
        gamma=gamma+1
#c=9 gamma=0.1

In [ ]:
data_credit_X_0=data_credit_X[data_credit_Y==0]
data_credit_X_1=data_credit_X[data_credit_Y==1]

In [ ]:
d_c_m_0=data_credit_X_0.mean(axis=0).reshape(1,30)
d_c_m_1=data_credit_X_1.mean(axis=0).reshape(1,30)

In [ ]:
Y=[0,1]

In [ ]:
line_1=np.array([d_c_m_0,d_c_m_1]).reshape(2,30)
Y_1=[1,0]

In [ ]:
li_1=svm.SVC(kernel='linear')
li_1.fit(line_1,Y_1)

In [ ]:
d_c_m_0

In [ ]:
d_c_m_1

In [ ]:
eq=li_1.coef_.reshape(30,1)

In [ ]:
deno=math.sqrt(np.sum(np.square(eq)))
dist_0=abs((np.matmul(data_credit_X_0,eq)+li_1.intercept_)/deno)

In [ ]:
len(dist_0[dist_0<0])

In [ ]:
dist_1=abs((np.matmul(data_credit_X_1,eq)+li_1.intercept_)/deno)

In [ ]:
len(dist_1[dist_1>0])

In [ ]:
in_d_1=np.argsort(dist_1,axis=0)

In [ ]:
in_d_1.shape

In [ ]:
in_d_0=np.argsort(dist_0,axis=0)

In [ ]:
in_d_0.shape

In [ ]:
X_0=data_credit_X_0[in_d_0]

In [ ]:
X_1=data_credit_X_1[in_d_1]

In [ ]:
x_1_tain=X_1

In [ ]:
x_0_tain=X_0[0:8000]

In [ ]:
len(x_0_tain)

In [ ]:
l=[0]*len(x_0_tain)+[1]*len(x_1_tain)

In [ ]:
len(l)

In [ ]:
l=np.array(l)

In [ ]:
l.shape

In [ ]:
k=np.append(x_0_tain,x_1_tain,axis=0)

In [ ]:
trian_X=k.reshape(8473,30)

In [ ]:
indices = np.arange(trian_X.shape[0]) 
np.random.shuffle(indices)

In [ ]:
trian_X=trian_X[indices]
l=l[indices]

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
tune_model=SVC(kernel='rbf', C=0.1, gamma=0.001,cache_size=2000, tol=1e-6, max_iter=1000)
# Fit the model on the training data
tune_model.fit(trian_X,l)
# Predict on the test data 
y_pred = tune_model.predict(credit_valid_X) 
# Generate the classification report 
report = classification_report(credit_valid_Y, y_pred,target_names=['0','1'])
#print(f"c : {C}  and gamma : {gamma}")
print("Classification Report:\n",report)

In [ ]:
confusion_matrix(credit_valid_Y,y_pred)

In [ ]:
from joblib import dump
dump(l12[4], 'tune_credit.joblib')

In [ ]:
test_Y=tune_model.predict(credit_test_X)

In [ ]:
report = classification_report(credit_test_Y, test_Y,target_names=['0','1'])
#print(f"c : {C}  and gamma : {gamma}")
print("Classification Report:\n",report)

In [ ]:
precision = precision_score(credit_test_Y, test_Y,labels=[0,1], average=None) 
recall = recall_score(credit_test_Y, test_Y,labels=[0,1], average=None) 
for label, precision in zip([0,1], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip([0,1], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
print("micro_F1")
print(f1_score(credit_test_Y, test_Y,labels=[0,1], average='micro'))
print("average F1")
print(f1_score(credit_test_Y, test_Y,labels=[0,1], average='weighted'))
print("macro F1")
print(f1_score(credit_test_Y, test_Y,labels=[0,1], average='macro')) 

print(f1_score(credit_test_Y, test_Y,labels=[0,1], average=None))


In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(credit_test_Y, test_Y).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:
from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(credit_test_Y,test_Y)
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
confusion_matrix(credit_test_Y,test_Y)

In [ ]:
trian_X.shape

In [ ]:
l=np.array(l)

In [ ]:
l.shape

In [ ]:
c : 1  and gamma : 0.01

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Define different values for C and gamma
C_values = [0.1, 1, 10]
gamma_values = [0.001, 0.01, 0.1]
l12=[]
j123=0
# Iterate over different combinations of C and gamma
for C in C_values:
    for gamma in gamma_values:
        # Create a SVC model
        smote_m=SVC(kernel='rbf', C=C, gamma=gamma,cache_size=2000, tol=1e-6, max_iter=1000)
        # Fit the model on the training data
        smote_m.fit(trian_X,l)
        print(j123)
        j123=j123+1
        l12.append(smote_m)
        # Predict on the test data 
        y_pred = smote_m.predict(credit_valid_X) 
        # Generate the classification report 
        report = classification_report(credit_valid_Y, y_pred,target_names=['0','1'])
        print(f"c : {C}  and gamma : {gamma}")
        print("Classification Report:\n",report)
        gamma=gamma+1

In [ ]:
'C': [0.1, 1, 10, 100],
'gamma': [0.001, 0.01, 0.1, 1]

In [ ]:
def distance(s_line,train_data,feature,intercept):
    deno=math.sqrt(np.sum(np.square(s_line)))
    dist_1=np.abs((np.matmul(train_data[:,0:feature],s_line)+intercept)/deno)
    return dist_1.flatten()

In [ ]:
def svm_im(traing_data_X=None,train_data_Y=None,clas=0,c1=0.50,cn1=0.50):
    train_data=np.copy(traing_data_X)
    #np.place(train_data[:, 768],train_data[:, 768]!=clas, [0])
    #np.place(train_data[:, 768],train_data[:, 768]==clas, [1])
    train_data_1= train_data[train_data_Y ==1 ]
    train_data_non_1=train_data[ train_data_Y!=1 ]
    length_1=train_data_1.shape[0]
    length_non_1=train_data_non_1.shape[0]
    feature=train_data_1.shape[1]
    center_1=train_data_1[:,:].mean(axis=0).reshape(1,feature)
    center_non_1=train_data_non_1[:,:].mean(axis=0).reshape(1,feature)
    line_1=np.array([center_1,center_non_1]).reshape(2,feature)
    Y_1=[1,0]
    li_1=svm.SVC(kernel='linear')
    li_1.fit(line_1,Y_1)
    d_1=distance(li_1.coef_[0].reshape(feature,1),train_data_1[:,0:feature],feature,li_1.intercept_)
    d_non_1=distance(li_1.coef_[0].reshape(feature,1),train_data_non_1[:,0:feature],feature,li_1.intercept_)
    sorted_indices_1 = np.argsort(d_1)
    t1=train_data_1[sorted_indices_1]
    sorted_indices_non_1=np.argsort(d_non_1)
    tn1=train_data_non_1[sorted_indices_non_1]
    l1=t1.shape[0]
    ln1=tn1.shape[0]
    y=[1]*(math.ceil(c1*l1))+[0]*math.ceil(cn1*ln1)
    y=np.array(y)
    t1=t1.reshape(l1,feature)
    tn1=tn1.reshape(ln1,feature)
    f_t= np.append(t1[0:math.ceil(c1*l1),:], tn1[0:math.ceil(cn1*ln1),:], axis=0)
    # Split data into batches
    indices = np.arange(f_t.shape[0]) 
    np.random.shuffle(indices)
    f_t=f_t[indices]
    y=y[indices]
    print(len(y[y==0]))
    print(len(y[y==1]))
    C_values = [0.1, 1, 10]
    gamma_values = [0.001, 0.01, 0.1]

    # Iterate over different combinations of C and gamma
    for C in C_values:
        for gamma in gamma_values:
            # Create a SVC model
            sp_1=SVC(kernel='rbf', C=C, gamma=gamma,cache_size=2000, tol=1e-3, max_iter=1000)
            # Fit the model on the training data
            sp_1.fit(f_t,y)
            # Predict on the test data 
            y_pred = sp_1.predict(credit_valid_X) 
            # Generate the classification report 
            report = classification_report(credit_valid_Y, y_pred,target_names=['0','1'])
            print(f"c : {C}  and gamma : {gamma}")
            print("Classification Report:\n",report)
    gc.collect()
    return 

In [ ]:
len(credit_train_Y[credit_train_Y==1])

In [ ]:
len(credit_train_Y[credit_train_Y==0])

In [ ]:
model1=svm_im(credit_train_X,credit_train_Y,clas=1,c1=1,cn1=0.0576)

our model ANN

In [ ]:
train_x,train_y=data_cre(traing_data_X=credit_train_X,train_data_Y=credit_train_Y,clas=1,c1=1,cn1=0.65)

In [ ]:
len(train_y[train_y==1])

In [ ]:
len(train_y[train_y==0])

In [ ]:
# Assuming senti_resample_Y is your target labels
# Convert your target labels to one-hot encoded format
train_Y_1 = to_categorical(train_y, num_classes=2)

# Similarly, convert your validation labels
valid_Y_1 = to_categorical(credit_valid_Y, num_classes=2)

# Define the model
model_co_1 = Sequential([
    Dense(15, input_dim=30, activation='relu'),
    Dense(8, activation='relu'),
    Dense(2, activation='softmax')  # Use softmax for multi-class classification
])

# Compile the model
model_co_1.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])

early_stopping = EarlyStopping(monitor='val_f1_score', patience=40, verbose=1, restore_best_weights=True)
# Train the model
history=model_co_1.fit(train_x,train_Y_1,epochs=100, batch_size=100, validation_data=(credit_valid_X,valid_Y_1), callbacks=[early_stopping])


In [ ]:
y_pred = model_co_1.predict(credit_valid_X).argmax(axis=1)
# Generate the classification report 
report = classification_report(credit_valid_Y, y_pred,target_names=['0','1'])


In [ ]:
print(report)

In [ ]:
y_t_pred = model_co_1.predict(credit_test_X).argmax(axis=1)
# Generate the classification report 
report = classification_report(credit_test_Y, y_t_pred,target_names=['0','1'])

print(report)

In [ ]:
model_co_1.save('our2_credit.h5')

In [ ]:
from keras.models import load_model
model_co_1 = load_model('/kaggle/input/our_credit_model/other/default/1/our2_credit.h5')

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

precision = precision_score(credit_test_Y, y_t_pred,labels=[0,1], average=None) 
recall = recall_score(credit_test_Y, y_t_pred,labels=[0,1], average=None) 
for label, precision in zip([0,1], precision):
    print(f'Precision for {label}: {precision:.4f}')
for label, recall in zip([0,1], recall):
    print(f'Recall for {label}: {recall:.4f}')

In [ ]:
micr=f1_score(credit_test_Y, y_t_pred,labels=[0,1], average='micro')
print(f"Micro f1 score:{micr}")
macr=f1_score(credit_test_Y, y_t_pred,labels=[0,1], average='macro')
print(f"Macro f1 score:{macr}")
weg=f1_score(credit_test_Y, y_t_pred,labels=[0,1], average='weighted')
print(f"Weight f1 score:{weg}")
a1,b1=f1_score(credit_test_Y, y_t_pred,labels=[0,1], average=None) 
print(f"0:{a1} 1:{b1}")

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(credit_test_Y, y_t_pred).ravel() 
# Calculate TPR and TNR 
tpr = tp / (tp + fn) 
tnr = tn / (tn + fp) 
# Calculate G-mean 
g_mean = np.sqrt(tpr * tnr) 
print(f'G-mean: {g_mean:.4f}')

In [ ]:

from sklearn.metrics import roc_curve, auc 
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(credit_test_Y,y_t_pred)
roc_auc = auc(fpr, tpr) 
print(f'AUC: {roc_auc:.2f}')

# Plot ROC curve 
plt.figure() 
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0]) 
plt.ylim([0.0, 1.05]) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.title('Receiver Operating Characteristic (ROC) Curve') 
plt.legend(loc='lower right') 
plt.show()

In [ ]:
trai=[]
val=[]

In [ ]:
for i1 in range(5,101,5):
    for j1 in range(100,101,5):
        i=i1/100
        j=j1/100
        train_x,train_y=data_cre(traing_data_X=credit_train_X,train_data_Y=credit_train_Y,clas=1,c1=j,cn1=i)
        # Assuming senti_resample_Y is your target labels
        # Convert your target labels to one-hot encoded format
        train_Y_1 = to_categorical(train_y, num_classes=2)

        # Similarly, convert your validation labels
        valid_Y_1 = to_categorical(credit_valid_Y, num_classes=2)

        # Define the model
        model_co_1 = Sequential([
            Dense(15, input_dim=30, activation='relu'),
            Dense(8, activation='relu'),
            Dense(2, activation='softmax')  # Use softmax for multi-class classification
        ])

        # Compile the model
        model_co_1.compile(optimizer=Adam(), 
              loss=CategoricalCrossentropy(), 
              metrics=[F1Score(average='macro')])

        early_stopping = EarlyStopping(monitor='val_f1_score', patience=9, verbose=1, restore_best_weights=True)
        # Train the model
        history=model_co_1.fit(train_x,train_Y_1,epochs=10, batch_size=100, validation_data=(credit_valid_X,valid_Y_1), callbacks=[early_stopping])
        y_pred = model_co_1.predict(credit_train_X).argmax(axis=1)
        macr_t=f1_score(credit_train_Y, y_pred,labels=[0,1], average='macro')
        y_pred = model_co_1.predict(credit_valid_X).argmax(axis=1)
        macr_v=f1_score(credit_valid_Y, y_pred,labels=[0,1], average='macro')
        trai.append([i,j,macr_t,macr_v])
        print(trai[-1])

In [ ]:
df_t = pd.DataFrame(trai, columns=['majority','minority','F1_macro_train','valid_macro_train'])

In [ ]:
df_t.to_excel('output_2.xlsx', index=False)

In [ ]:
val

In [ ]:
trin=[]
val=[]
for i in range(100,101,5):
    for j in range(5,101,5):
        class_1=i/100
        class_n_1=j/100
        train_x,train_y=data_cre(traing_data_X=credit_train_X,train_data_Y=credit_train_Y,clas=1,c1=class_1,cn1=class_n_1)
        print(len(train_y[train_y==1]))
        print(len(train_y[train_y==0]))
        # Define the model
        sp_1=SVC(kernel='rbf', C=0.1, gamma=0.001,cache_size=2000, tol=1e-3, max_iter=1000)
        # Fit the model on the training data
        sp_1.fit(train_x,train_y)
        y_1=sp_1.predict(credit_train_X)
        macro_f1_1 = f1_score(credit_train_Y, y_1, average='macro')
        trin.append([class_1,class_n_1,macro_f1_1])
        y_1=sp_1.predict(credit_valid_X)
        macro_f1_2 = f1_score(credit_valid_Y, y_1, average='macro')
        val.append([class_1,class_n_1,macro_f1_2])
        print(f"c1:{class_1} cn1:{class_n_1} test:{macro_f1_1} valid:{macro_f1_2}")

In [ ]:
trin=[]
val=[]
j=5.76451
for i in range(5,101,5):
        class_1=i/100
        class_n_1=j/100
        train_x,train_y=data_cre(traing_data_X=credit_train_X,train_data_Y=credit_train_Y,clas=1,c1=class_1,cn1=class_n_1)
        print(len(train_y[train_y==1]))
        print(len(train_y[train_y==0]))
        # Define the model
        sp_1=SVC(kernel='rbf', C=0.1, gamma=0.001,cache_size=2000, tol=1e-3, max_iter=1000)
        # Fit the model on the training data
        sp_1.fit(train_x,train_y)
        y_1=sp_1.predict(credit_train_X)
        macro_f1_1 = f1_score(credit_train_Y, y_1, average='macro')
        trin.append([class_1,class_n_1,macro_f1_1])
        y_1=sp_1.predict(credit_valid_X)
        macro_f1_2 = f1_score(credit_valid_Y, y_1, average='macro')
        val.append([class_1,class_n_1,macro_f1_2])
        print(f"c1:{class_1} cn1:{class_n_1} test:{macro_f1_1} valid:{macro_f1_2}")

In [ ]:
trin=[]
val=[]
for i in range(5,101,5):
    for j in range(5,101,5):
        class_1=i/100
        class_n_1=j/100
        train_x,train_y=data_cre(traing_data_X=credit_train_X,train_data_Y=credit_train_Y,clas=1,c1=class_1,cn1=class_n_1)
        print(len(train_y[train_y==1]))
        print(len(train_y[train_y==0]))
        # Define the model
        sp_1=SVC(kernel='rbf', C=0.1, gamma=0.001,cache_size=2000, tol=1e-3, max_iter=1000)
        # Fit the model on the training data
        sp_1.fit(train_x,train_y)
        y_1=sp_1.predict(credit_train_X)
        macro_f1_1 = f1_score(credit_train_Y, y_1, average='macro')
        trin.append([class_1,class_n_1,macro_f1_1])
        y_1=sp_1.predict(credit_valid_X)
        macro_f1_2 = f1_score(credit_valid_Y, y_1, average='macro')
        val.append([class_1,class_n_1,macro_f1_2])
        print(f"c1:{class_1} cn1:{class_n_1} test:{macro_f1_1} valid:{macro_f1_2}")

In [ ]:
t1=np.array(trin)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Generate some example validation data
majority_class_percen = t1[:,0:1]
minority_class_percen = t1[:,1:2]
macro_f1=t1[:,2:3]  # Example loss values for demonstration
print(len(macro_f1))
# Create a meshgrid for 3D plotting
X, Y = np.meshgrid(majority_class_percen, minority_class_percen)
Z = macro_f1

# Create a 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Add labels and title
ax.set_xlabel('majority_class_percen')
ax.set_ylabel('minority_class_percen')
ax.set_zlabel('macro_f1')
ax.set_title('3D train prediction')

# Add a color bar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Show the plot
plt.show()

In [ ]:
t1=np.array(val)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Generate some example validation data
majority_class_percen = t1[:,0:1]
minority_class_percen = t1[:,1:2]
macro_f1=t1[:,2:3]  # Example loss values for demonstration
print(len(macro_f1))
# Create a meshgrid for 3D plotting
X, Y = np.meshgrid(majority_class_percen, minority_class_percen)
Z = macro_f1

# Create a 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Add labels and title
ax.set_xlabel('majority_class_percen')
ax.set_ylabel('minority_class_percen')
ax.set_zlabel('macro_f1')
ax.set_title('3D validation prediction')

# Add a color bar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Show the plot
plt.show()